In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

fa = pd.read_excel(r"C:\Users\Lara\Documents\Ironhack\Course\Mini Projects\Mini project 3\febre_amarela.xlsx")

In [2]:
fa.drop(columns=["COD_MUN_LPI", "MUN_LPI", "DT_IS", "SE_IS", "MES_IS", "MONITORAMENTO_IS", "DT_OBITO"], inplace=True)

In [3]:
fa.rename(columns= {"MACRORREG_LPI":"region","COD_UF_LPI":"code","UF_LPI":"state","SEXO":"sex","IDADE":"age","ANO_IS":"date","OBITO":"death" }, inplace = True)
fa.columns = [col.lower() for col in fa.columns]
fa["death"].replace("SIM","yes", inplace = True)
fa["death"].replace("NÃO","no", inplace = True)
fa["death"].replace("IGN","unk", inplace = True)
fa = fa[(fa["date"] >= 2010) & (fa["date"] < 2020)]
fa["date"] = fa["date"].astype(str)

In [4]:
fa.dropna()

,id,region,code,state,sex,age,date,death
456,457,CO,50,MS,M,39.0,2010,yes
457,458,N,15,PA,M,31.0,2010,yes
458,459,N,15,PA,M,19.0,2011,no
459,460,N,15,PA,M,15.0,2011,yes
460,461,N,13,AM,M,17.0,2013,no
...,...,...,...,...,...,...,...,...
2724,2725,N,15,PA,M,57.0,2019,yes
2725,2726,N,15,PA,M,18.0,2019,yes
2726,2727,S,42,SC,M,40.0,2019,yes
2727,2728,N,13,AM,M,21.0,2019,no


In [5]:
coo_cap={
    "AC": [ -8.77, -70.55]
  , "AM": [ -3.07, -61.66]
  , "AP": [  1.41, -51.77]
  , "BA": [-12.96, -38.51]
  , "DF": [-15.83, -47.86]
  , "ES": [-19.19, -40.34]
  , "GO": [-16.64, -49.31]
  , "MA": [ -2.55, -44.30]
  , "MT": [-12.64, -55.42]
  , "MS": [-20.51, -54.54]
  , "MG": [-18.10, -44.38]
  , "PA": [ -5.53, -52.29]
  , "PR": [-24.89, -51.55]
  , "RJ": [-22.84, -43.15]
  , "RO": [-11.22, -62.80]
  , "RS": [-30.01, -51.22]
  , "RR": [  1.89, -61.22]
  , "SC": [-27.33, -49.44]
  , "SP": [-23.55, -46.64]
  , "TO": [-10.25, -48.25]
}

In [6]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": -8.77,
    "longitude": -70.55,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_am = pd.DataFrame(data = daily_data)

Coordinates -8.752196311950684°N -70.52587890625°E
Elevation 233.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [7]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-3.07,
    "longitude": -61.66,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_ac = pd.DataFrame(data = daily_data)

Coordinates -3.0579965114593506°N -61.64385986328125°E
Elevation 31.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [8]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":1.41,
    "longitude": -51.77,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_ap = pd.DataFrame(data = daily_data)

Coordinates 1.441124677658081°N -51.764251708984375°E
Elevation 158.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [9]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-12.96,
    "longitude": -38.51,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_ba = pd.DataFrame(data = daily_data)

Coordinates -12.970122337341309°N -38.618194580078125°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [10]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-15.83,
    "longitude": -47.86,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_df = pd.DataFrame(data = daily_data)

Coordinates -15.852372169494629°N -47.847015380859375°E
Elevation 1010.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [11]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-19.19,
    "longitude": -40.34,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_es = pd.DataFrame(data = daily_data)

Coordinates -19.156415939331055°N -40.37548828125°E
Elevation 152.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [12]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-16.64,
    "longitude": -49.31,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_go = pd.DataFrame(data = daily_data)

Coordinates -16.625659942626953°N -49.29388427734375°E
Elevation 743.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [13]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-2.55,
    "longitude": -44.30,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_ma = pd.DataFrame(data = daily_data)

Coordinates -2.5659050941467285°N -44.278839111328125°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [14]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-12.64,
    "longitude": -55.42,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_mt = pd.DataFrame(data = daily_data)

Coordinates -12.61862850189209°N -55.384613037109375°E
Elevation 402.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [15]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-20.51,
    "longitude": -54.54,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_ms = pd.DataFrame(data = daily_data)

Coordinates -20.49209213256836°N -54.561920166015625°E
Elevation 601.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [16]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-18.10,
    "longitude": -44.38,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_mg = pd.DataFrame(data = daily_data)

Coordinates -18.1019344329834°N -44.430389404296875°E
Elevation 970.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [17]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-5.53,
    "longitude": -52.29,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_pa = pd.DataFrame(data = daily_data)

Coordinates -5.518453121185303°N -52.313446044921875°E
Elevation 199.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [18]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-24.89,
    "longitude": -51.55,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_pr = pd.DataFrame(data = daily_data)

Coordinates -24.920913696289062°N -51.580657958984375°E
Elevation 914.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [19]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-22.84,
    "longitude": -43.15,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_rj = pd.DataFrame(data = daily_data)

Coordinates -22.81195068359375°N -43.03125°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [20]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-11.22,
    "longitude": -62.80,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_ro = pd.DataFrame(data = daily_data)

Coordinates -11.212653160095215°N -62.800018310546875°E
Elevation 276.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [21]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-30.01,
    "longitude": -51.22,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_rs = pd.DataFrame(data = daily_data)

Coordinates -29.982425689697266°N -51.188812255859375°E
Elevation 6.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [22]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":1.89,
    "longitude": -61.22,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_rr = pd.DataFrame(data = daily_data)

OpenMeteoRequestsError: {'reason': 'Minutely API request limit exceeded. Please try again in one minute.', 'error': True}

In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-27.33,
    "longitude": -49.44,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_sc = pd.DataFrame(data = daily_data)

In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-23.55,
    "longitude": -46.64,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_sp = pd.DataFrame(data = daily_data)

In [ ]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude":-10.25,
    "longitude": -48.25,
    "start_date": "2010-01-01",
    "end_date": "2019-12-31",
    "daily": ["temperature_2m_mean", "precipitation_sum", "precipitation_hours"]
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive = "left"
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe_to = pd.DataFrame(data = daily_data)

In [ ]:
dflist=[daily_dataframe_ac,daily_dataframe_am,daily_dataframe_ap,daily_dataframe_ba,daily_dataframe_df,daily_dataframe_es,daily_dataframe_go,daily_dataframe_ma,daily_dataframe_mt,daily_dataframe_ms,daily_dataframe_mg,daily_dataframe_pa,daily_dataframe_pr,daily_dataframe_rj,daily_dataframe_ro,daily_dataframe_rs,daily_dataframe_rr,daily_dataframe_sc,daily_dataframe_sp,daily_dataframe_to]

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# "AC" =="Acre"
# "AM" == "Amazonas"
# "AP" == "Amapá"
# "BA" == "Bahia"
# "DF" == "Distrito Federal"
# "ES" == "Espírito Santo"
# "GO" == "Goiás"
# "MA" == "Maranhão"
# "MT" == "Mato Grosso"
# "MS" == "Mato Grosso do Sul"
# "MG" == "Minas Gerais"
# "PA" == "Pará"
# "PR" == "Paraná"
# "RJ" == "Rio de Janeiro"
# "RS" == "Rio Grande do Sul"
# "RO" == "Rondônia"
# "RR" == "Roraima"
# "SC" == "Santa Catarina"
# "SP" == "São Paulo"
# "TO" == "Tocantins"

In [ ]:
for df in dflist:
    df ["date"] = df['date'].astype(str)   

In [ ]:
years = range(2010, 2020)  # Define the range of years

for year in years:
    # Filter and assign the year value to 'date' column for each year
    filtered_ac = daily_dataframe_ac.loc[daily_dataframe_ac['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_ac['date'].str.contains("^" + str(year))
    daily_dataframe_ac.loc[filtered_rows, 'date'] = str(year)

for year in years:
    filtered_am = daily_dataframe_am.loc[daily_dataframe_am['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_am['date'].str.contains("^" + str(year))
    daily_dataframe_am.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_ap = daily_dataframe_ap.loc[daily_dataframe_ap['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_ap['date'].str.contains("^" + str(year))
    daily_dataframe_ap.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_ba = daily_dataframe_ba.loc[daily_dataframe_ba['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_ba['date'].str.contains("^" + str(year))
    daily_dataframe_ba.loc[filtered_rows, 'date'] = str(year)
       
for year in years:
    filtered_ba = daily_dataframe_df.loc[daily_dataframe_df['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_df['date'].str.contains("^" + str(year))
    daily_dataframe_df.loc[filtered_rows, 'date'] = str(year)

for year in years:
    filtered_es = daily_dataframe_es.loc[daily_dataframe_es['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_es['date'].str.contains("^" + str(year))
    daily_dataframe_es.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_go = daily_dataframe_go.loc[daily_dataframe_go['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_go['date'].str.contains("^" + str(year))
    daily_dataframe_go.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_ma = daily_dataframe_ma.loc[daily_dataframe_ma['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_ma['date'].str.contains("^" + str(year))
    daily_dataframe_ma.loc[filtered_rows, 'date'] = str(year)

for year in years:
    filtered_mt = daily_dataframe_mt.loc[daily_dataframe_mt['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_mt['date'].str.contains("^" + str(year))
    daily_dataframe_mt.loc[filtered_rows, 'date'] = str(year)

for year in years:
    filtered_ms = daily_dataframe_ms.loc[daily_dataframe_ms['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_ms['date'].str.contains("^" + str(year))
    daily_dataframe_ms.loc[filtered_rows, 'date'] = str(year)

for year in years:
    filtered_mg = daily_dataframe_mg.loc[daily_dataframe_mg['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_mg['date'].str.contains("^" + str(year))
    daily_dataframe_mg.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_pa = daily_dataframe_pa.loc[daily_dataframe_pa['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_pa['date'].str.contains("^" + str(year))
    daily_dataframe_pa.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_pr = daily_dataframe_pr.loc[daily_dataframe_pr['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_pr['date'].str.contains("^" + str(year))
    daily_dataframe_pr.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_rj = daily_dataframe_rj.loc[daily_dataframe_rj['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_rj['date'].str.contains("^" + str(year))
    daily_dataframe_rj.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_ro = daily_dataframe_ro.loc[daily_dataframe_ro['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_ro['date'].str.contains("^" + str(year))
    daily_dataframe_ro.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_rs = daily_dataframe_rs.loc[daily_dataframe_rs['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_rs['date'].str.contains("^" + str(year))
    daily_dataframe_rs.loc[filtered_rows, 'date'] = str(year)
        
for year in years:
    filtered_rr = daily_dataframe_rr.loc[daily_dataframe_rr['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_rr['date'].str.contains("^" + str(year))
    daily_dataframe_rr.loc[filtered_rows, 'date'] = str(year)
      
for year in years:
    filtered_sc = daily_dataframe_sc.loc[daily_dataframe_sc['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_sc['date'].str.contains("^" + str(year))
    daily_dataframe_sc.loc[filtered_rows, 'date'] = str(year)
    
for year in years:
    filtered_sp = daily_dataframe_sp.loc[daily_dataframe_sp['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_sp['date'].str.contains("^" + str(year))
    daily_dataframe_sp.loc[filtered_rows, 'date'] = str(year)

for year in years:
    filtered_to = daily_dataframe_to.loc[daily_dataframe_to['date'].str.contains(str(year)), "date"] = str(year)
    filtered_rows = daily_dataframe_to['date'].str.contains("^" + str(year))
    daily_dataframe_to.loc[filtered_rows, 'date'] = str(year)

In [ ]:
grouped_ac=daily_dataframe_ac.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_ac=grouped_ac.applymap(lambda x:round(x,2))

grouped_am=daily_dataframe_am.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_am=grouped_ac.applymap(lambda x:round(x,2))

grouped_ap=daily_dataframe_ap.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_ap=grouped_ap.applymap(lambda x:round(x,2))

grouped_ba=daily_dataframe_ba.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_ba=grouped_ba.applymap(lambda x:round(x,2))

grouped_df=daily_dataframe_df.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_df=grouped_df.applymap(lambda x:round(x,2))

grouped_es=daily_dataframe_es.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_es=grouped_es.applymap(lambda x:round(x,2))

grouped_go=daily_dataframe_go.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_go=grouped_go.applymap(lambda x:round(x,2))

grouped_ma=daily_dataframe_ma.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_ma=grouped_ma.applymap(lambda x:round(x,2))

grouped_mt=daily_dataframe_mt.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_mt=grouped_mt.applymap(lambda x:round(x,2))

grouped_ms=daily_dataframe_ms.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_ms=grouped_ms.applymap(lambda x:round(x,2))

grouped_mg=daily_dataframe_mg.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_mg=grouped_mg.applymap(lambda x:round(x,2))

grouped_pa=daily_dataframe_pa.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_pa=grouped_pa.applymap(lambda x:round(x,2))

grouped_pr=daily_dataframe_pr.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_pr=grouped_pr.applymap(lambda x:round(x,2))

grouped_rj=daily_dataframe_rj.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_rj=grouped_rj.applymap(lambda x:round(x,2))

grouped_rs=daily_dataframe_rs.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_rs=grouped_rs.applymap(lambda x:round(x,2))

grouped_ro=daily_dataframe_ro.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_ro=grouped_ro.applymap(lambda x:round(x,2))

grouped_rr=daily_dataframe_rr.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_rr=grouped_rr.applymap(lambda x:round(x,2))

grouped_sc=daily_dataframe_sc.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_sc=grouped_sc.applymap(lambda x:round(x,2))

grouped_sp=daily_dataframe_sp.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_sp=grouped_sp.applymap(lambda x:round(x,2))

grouped_to=daily_dataframe_to.groupby(["date"]).agg({"temperature_2m_mean":"mean","precipitation_sum":"mean","precipitation_hours":"mean"})
grouped_to=grouped_to.applymap(lambda x:round(x,2))

In [ ]:
grouped_ac["state"] = "AC"
grouped_am["state"] = "AM"
grouped_ap["state"] = "AP"
grouped_ba["state"] = "BA"
grouped_df["state"] = "DF"
grouped_es["state"] = "ES"
grouped_go["state"] = "GO"
grouped_ma["state"] = "MA"
grouped_mt["state"] = "MT"
grouped_ms["state"] = "MS"
grouped_mg["state"] = "MG"
grouped_pa["state"] = "PA"
grouped_pr["state"] = "PR"
grouped_rj["state"] = "RJ"
grouped_ro["state"] = "RO"
grouped_rs["state"] = "RS"
grouped_rr["state"] = "RR"
grouped_sc["state"] = "SC"
grouped_sp["state"] = "SP"
grouped_to["state"] = "TO"

In [ ]:
grouped_ac.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_am.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_ap.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_ba.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_df.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_es.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_go.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_ma.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_mt.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_ms.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_mg.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_pa.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_pr.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_rj.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_rs.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_ro.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_rr.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_sc.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_sp.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)
grouped_to.rename(columns= {"temperature_2m_mean":"temperature","precipitation_sum":"precipitation","precipitation_hours":"hours"}, inplace = True)

In [ ]:
meteo = pd.concat([grouped_ac,grouped_am,grouped_ap,grouped_ba, grouped_df, grouped_es, grouped_go, grouped_ma, grouped_mt, grouped_ms,grouped_mg, grouped_pa, grouped_pr, grouped_rj, grouped_rs, grouped_ro, grouped_rr, grouped_sc, grouped_sp, grouped_to])

In [ ]:
meteo.reset_index(inplace = True)

In [ ]:
value_counts_per_state_date = fa.groupby(['state', 'date']).size()
cases = pd.DataFrame(value_counts_per_state_date )

In [ ]:
cases.reset_index(inplace = True)

In [ ]:
fa_pivot = pd.pivot_table(fa, index = fa["date"], columns = fa["state"], values = "death",aggfunc='count', fill_value=0).reset_index()

In [ ]:
fa_pivot

In [ ]:
fa_pivot.columns

In [ ]:
import numpy as np
fap2012 = np.array([2012,0,0,0,0,0,0,0,0,0,0,0,0,0])
fap2012=pd.DataFrame([fap2012],columns=fa_pivot.columns)
fa_pivot=pd.concat([fa_pivot,fap2012])

In [ ]:
fa_pivot

In [ ]:
fa_final = pd.melt(fa_pivot, id_vars = "date")
fa_final

In [ ]:
meteo.dtypes

In [ ]:
fa_final.dtypes

In [ ]:
final = pd.merge(fa_final, meteo, on=["state","date"], how="outer")

In [ ]:
for state in final["state"]:
    final["temperature"].fillna(final["temperature"].mean(), inplace=True)
    final["precipitation"].fillna(final["precipitation"].mean(), inplace=True)
    final["hours"].fillna(final["hours"].mean(), inplace=True)

In [ ]:
final["date"] = final["date"].astype(int)

In [ ]:
final["temperature"] = final["temperature"].apply(lambda x:round(x,2))
final["precipitation"] = final["precipitation"].apply(lambda x:round(x,2))
final["hours"] = final["hours"].apply(lambda x:round(x,2))

In [ ]:
final.rename(columns= {"date": "year"}, inplace=True)

In [ ]:
final.to_csv('data.csv')